In [1]:
import obstools as ot
from pathlib import Path
import json
import networkx as nx

class VaultExhaustive:
    def __init__(self, vault_dir = None, file_name = None, make_json = 0.5):
        if not vault_dir is None:
            self.vault_dir = Path(vault_dir)
            self.all_paths = ot.get_all_paths(vault_dir)
            self.names_to_paths = ot.get_names_to_paths(self.all_paths)
            names_to_links_results = ot.get_names_to_links(self.names_to_paths)
            self.names_to_links = names_to_links_results[0]
            self.reading_errors = names_to_links_results[1]
            self.json = None
            if not file_name is None:
                self.file_name = Path(file_name)
                if make_json == 0.5:
                    if self.file_name.is_file():
                        try: 
                            with open(self.file_name, 'r') as ofile:
                                self.json = json.load(ofile)
                            make_json = False
                        except:
                            make_json = True
                    else:
                        make_json = True
                if make_json:
                    self.dump_errors = ot.produce_json_of_vault(self.vault_dir, self.file_name)
                    with open(self.file_name, 'r') as ofile:
                        self.json = json.load(ofile)
            self.mapping = dict()
            if type(self.json) is dict:
                self.mapping = ot.get_mapping_from_json(self.json)

v = VaultExhaustive("C://Users//zrfir//OneDrive//__Obsidian//PUBLIC", "2022-02-02_Obsidian_2.json", 1)

In [3]:
from copy import deepcopy
def f(values):
    o = 0
    for v in values:
        if v > o:
            o = v
    return o + 1
    

def scalp_self_mapping(dictionary, rsr=True, firstly=True):
    """
    This method will take a self-mapping dictionary and begin stripping leaves. It will remove all leaves and
    place this as a set into a list[0]. It will repeat, putting all newly made leaves as a result of the previous
    scalping into the next list[1], each time abolishing leaves from existing in the scaped dictionary.
    And so on until list[n], when leaves are present.
    
    Returns a pair where pair[0] is the final scalped dictionary, and pair[1] is the list of sets of leaves.
    
    @param dictionary: self-mapping dictionary
    """
    def helper(d):
        """
        This helper method removes leaves and returns a scalped dictionary and the set of leaves.
        return[0] is the new scalped dictionary (a copy)
        return[1] is a list of the newly scalped leaves
        """
        if type(d) is dict:
            d1 = deepcopy(d)
            o = set()
            for key, values in d.items():
                if len(values) == 0:
                    for key0, values0 in d1.items():
                        if key in values0:
                            values0.remove(key)
                    d1.pop(key)
                    o.add(key)
            return [d1,o]
        elif type(d) is list:
            return helper(d[0])
    o = list()
    if rsr and firstly:
        dictionary = remove_self_referencing(dictionary)
    # scalp for the first time to get a dictionary[1] results
    dictionary = helper(dictionary)
    o.append(dictionary[1])
    # continue until dictionary[1] results are empty (no new leaves to scalp)
    while len(dictionary[1]) > 0:
        dictionary = helper(dictionary)
        o.append(dictionary[1])
    if rsr and not firstly:
        dictionary[0] = remove_self_referencing(dictionary[0])
    return [dictionary[0],o]

def remove_self_referencing(dictionary, copy=True):
    if copy:
        dictionary = deepcopy(dictionary)
    for key, values in dictionary:
        if key in values:
            values.remove(key)
    return dictionary    

answers = dict()
scalped = scalp(v.mapping)
s = scalped[1]
for name in s[0]:
    answers[name] = 0
for i in range(1,len(s)):
    for name in s[i]:
        values = set()
        for dep in v.mapping[name]:
            values.add(answers[dep])
        answers[name] = f(values)
# answers
# ot.get_backlinks(answers)

In [7]:
d = {
    1: {1,2,3},
    2: {5,6,7},
}
e = ot.get_backlinks(d)
f = ot.get_backlinks(e)
f

{1: {1, 2, 3}, 2: {5, 6, 7}}

In [ ]:
ot.